This is a market cap weighted portfolio using the holdings of the Russell 2000 ETF (IWM)

In [1]:
#import packages
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

In [2]:
#read in the tickers
stocks = pd.read_csv('russ_tickers.csv')

In [3]:
#check the dataframe
stocks

,Ticker
0,OVV
1,MIME
2,FHI
3,GT
4,CIT
...,...
1951,ZIOP
1952,ZIXI
1953,ZUMZ
1954,ZUO


In [4]:
#this is a common Sandbox API for the IEX Cloud
#It will pull in stale data
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

In [5]:
#create columns for the data frame
columns = ['Ticker', 'Company Name','Price','Market Capitalization', 'Weight',
           'Number Of Shares to Buy', 'Position Value']

In [6]:
#This function will break the list of tickers into smaller lists
#The API works quicker when this is are broken into smaller pieces
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [7]:
#Retreive the data from the API
symbol_groups = list(chunks(stocks['Ticker'], 10))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))


df = pd.DataFrame(columns = columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        df = df.append(
                                        pd.Series([symbol, data[symbol]['quote']['companyName'],
                                                   data[symbol]['quote']['latestPrice'], 
                                                   data[symbol]['quote']['marketCap'], "N/A","N/A","N/A"], 
                                                  index = columns), 
                                        ignore_index = True)
df

,Ticker,Company Name,Price,Market Capitalization,Weight,Number Of Shares to Buy,Position Value
0,OVV,Ovintiv Inc,23.710,6216726921,N/A,N/A,N/A
1,MIME,Mimecast Ltd,43.090,2810390651,N/A,N/A,N/A
2,FHI,Federated Hermes Inc - Class B,31.190,3124941061,N/A,N/A,N/A
3,GT,Goodyear Tire & Rubber Co.,18.010,4176345625,N/A,N/A,N/A
4,CIT,CIT Group Inc,54.300,5392145532,N/A,N/A,N/A
...,...,...,...,...,...,...,...
1951,ZIOP,"Ziopharm Oncology, Inc.",3.470,744996273,N/A,N/A,N/A
1952,ZIXI,Zix Corp.,7.752,456548295,N/A,N/A,N/A
1953,ZUMZ,Zumiez Inc,47.300,1172182573,N/A,N/A,N/A
1954,ZUO,Zuora Inc - Class A,17.820,1906454985,N/A,N/A,N/A


In [8]:
#Function that will ask for the dollar size of the portfolio to build.
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

In [9]:
portfolio_input()

Enter the value of your portfolio:10000000


In [10]:
#Calculate the weights for each ticker based on market cap
df['Weight'] = ((df['Market Capitalization']/df['Market Capitalization'].sum())*100).astype(float)
df.Weight= df.Weight.round(2)
#calculate the number of shares to buy
for i in range(0, len(df['Ticker'])):
    df.loc[i, 'Number Of Shares to Buy'] = round(((float(portfolio_size) * (df['Weight'][i])/100))/(df['Price'][i]),0)
df

,Ticker,Company Name,Price,Market Capitalization,Weight,Number Of Shares to Buy,Position Value
0,OVV,Ovintiv Inc,23.710,6216726921,0.18,759,N/A
1,MIME,Mimecast Ltd,43.090,2810390651,0.08,186,N/A
2,FHI,Federated Hermes Inc - Class B,31.190,3124941061,0.09,289,N/A
3,GT,Goodyear Tire & Rubber Co.,18.010,4176345625,0.12,666,N/A
4,CIT,CIT Group Inc,54.300,5392145532,0.16,295,N/A
...,...,...,...,...,...,...,...
1951,ZIOP,"Ziopharm Oncology, Inc.",3.470,744996273,0.02,576,N/A
1952,ZIXI,Zix Corp.,7.752,456548295,0.01,129,N/A
1953,ZUMZ,Zumiez Inc,47.300,1172182573,0.03,63,N/A
1954,ZUO,Zuora Inc - Class A,17.820,1906454985,0.06,337,N/A


In [11]:
#calculate the value of each position
df['Position Value'] = df['Number Of Shares to Buy']*df['Price']

In [12]:
df

,Ticker,Company Name,Price,Market Capitalization,Weight,Number Of Shares to Buy,Position Value
0,OVV,Ovintiv Inc,23.710,6216726921,0.18,759,17995.9
1,MIME,Mimecast Ltd,43.090,2810390651,0.08,186,8014.74
2,FHI,Federated Hermes Inc - Class B,31.190,3124941061,0.09,289,9013.91
3,GT,Goodyear Tire & Rubber Co.,18.010,4176345625,0.12,666,11994.7
4,CIT,CIT Group Inc,54.300,5392145532,0.16,295,16018.5
...,...,...,...,...,...,...,...
1951,ZIOP,"Ziopharm Oncology, Inc.",3.470,744996273,0.02,576,1998.72
1952,ZIXI,Zix Corp.,7.752,456548295,0.01,129,1000.01
1953,ZUMZ,Zumiez Inc,47.300,1172182573,0.03,63,2979.9
1954,ZUO,Zuora Inc - Class A,17.820,1906454985,0.06,337,6005.34


In [13]:
#write the dataframe to an Excel file
writer = pd.ExcelWriter('trades.xlsx', engine='xlsxwriter')
final_df = df
final_df.to_excel(writer, sheet_name='Trades', index = False)
final_df

,Ticker,Company Name,Price,Market Capitalization,Weight,Number Of Shares to Buy,Position Value
0,OVV,Ovintiv Inc,23.710,6216726921,0.18,759,17995.9
1,MIME,Mimecast Ltd,43.090,2810390651,0.08,186,8014.74
2,FHI,Federated Hermes Inc - Class B,31.190,3124941061,0.09,289,9013.91
3,GT,Goodyear Tire & Rubber Co.,18.010,4176345625,0.12,666,11994.7
4,CIT,CIT Group Inc,54.300,5392145532,0.16,295,16018.5
...,...,...,...,...,...,...,...
1951,ZIOP,"Ziopharm Oncology, Inc.",3.470,744996273,0.02,576,1998.72
1952,ZIXI,Zix Corp.,7.752,456548295,0.01,129,1000.01
1953,ZUMZ,Zumiez Inc,47.300,1172182573,0.03,63,2979.9
1954,ZUO,Zuora Inc - Class A,17.820,1906454985,0.06,337,6005.34


In [14]:
#formating for the Excel sheet
background_color = 'white'
font_color = 'black'


string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$#,#.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_format = writer.book.add_format(
        {
            'num_format':'0.00%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'#,#00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [15]:
#formate the Excel columns
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Company Name', string_format],
                    'C': ['Price', dollar_format],
                    'D': ['Market Capitalization', dollar_format],
                    'E': ['Weight', percent_format],
                    'F': ['Number of Shares to Buy', integer_format],
                    'G': ['Position Value', dollar_format]
                    }

In [16]:
for column in column_formats.keys():
    writer.sheets['Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Trades'].write(f'{column}1', column_formats[column][0], string_format)

In [17]:
writer.save()